In [319]:
## Let's try to figure out who the shortest, heaviest baseball players are.

## Here's our overall plan of attack

# Setting Things Up ✅
## Import CSVs ✅
### Separate CSVs --> DataFrames for People, Pitching Data, Batting Data ✅
## Squish everything into one mondo DF ✅
## Add Physical Data ✅
### Height ✅
### Weight ✅
## Calculate BMI ✅
### Convert Imperial to Metric ✅
### BMI-ify ✅
### Throw BMI back into df ✅
## Calculate mean fWAR/bWAR (mWAR)
## Assemble per-position lists sorted by BMI, then mWAR

# Knocking Things Down
## Find worst team that made playoffs in 2021
### Describe team fWAR/bWAR
### Describe individual fWAR/bWAR
## Pull from BMI lists per position until high BMI roster is full
### mWAR shall be higher on a team basis.
### mWAR shall be higher per position.
# If there's enough time:
## Repeat for:
### Tallness
### Shortness
### Heavy
### Light

# If there's enough time:
## Repeat for:
### Tallness
### Shortness
### Heavy
### Light

## Setting Things Up

### Import the necessaries

In [320]:
import pandas as pd
import pybaseball
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


In [321]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254

In [322]:
chadwick = chadwick_register(save=True)
chadwick = pd.DataFrame(chadwick)

In [323]:
download_lahman()

## Read infinity baseball data to DataFrames

In [324]:
# a table of all player biographical info and ids
people = pd.DataFrame(people())

# park id, name, alias, city, state, and country
parks = pd.DataFrame(parks())

# all star roster data: player, year, team, league, position
allstar = pd.DataFrame(all_star_full())

# each player's games played per position for each season
appearances = pd.DataFrame(appearances())

# batting stats by year, regular season
batting = pd.DataFrame(batting())

# batting stats by year, post season
batting_post = pd.DataFrame(batting_post())

# fielding stats by year 
fielding = pd.DataFrame(fielding())

# games played in left, center, right field 
fielding_of = pd.DataFrame(fielding_of())

# LF/CF/RF splits
fielding_of_split = pd.DataFrame(fielding_of_split())

# postseason fielding 
fielding_post = pd.DataFrame(fielding_post())

# home game attendance by park by year 
home_games = pd.DataFrame(home_games())

# historical player pitching stats
pitching = pd.DataFrame(pitching())

# postseason pitching stats
pitching_post = pd.DataFrame(pitching_post())

# playoff series winners and losers 
series_post = pd.DataFrame(series_post())

# data on teams by year: record, division, stadium, attendance, etc
teams = pd.DataFrame(teams())

# current and historical franchises, whether they're still active, and their ids
teams_franchises = pd.DataFrame(teams_franchises())

# split season data for teams
teams_half = pd.DataFrame(teams_half()) 

# fangraphs batting since 2008
fangraphs_batting = pd.DataFrame(pybaseball.batting_stats_range(
    start_dt="2008-01-01", end_dt="2021-12-31"))

# fangraphs pitching since 2008
fangraphs_pitching = pd.DataFrame(pybaseball.pitching_stats_range(
    start_dt="2008-01-01", end_dt="2021-12-31"))

# fangraphs team pitching since 2008
fangraphs_team_pitching = pd.DataFrame(
    pybaseball.team_pitching(start_season="2008", end_season="2021"))

# fangraphs team batting since 2008
fangraphs_team_batting = pd.DataFrame(
    pybaseball.team_batting(start_season="2008", end_season="2021"))

# fangraphs team fielding since 2008
fangraphs_team_fielding = pd.DataFrame(
    pybaseball.team_fielding(start_season="2008", end_season="2021"))

# bref pitching WAR
bwar_pitch = pd.DataFrame(bwar_pitch(return_all=True))

# bref batting WAR
bwar_bat = pd.DataFrame(bwar_bat(return_all=True))


/Users/bean/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pybaseball/league_batting_stats.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


Here's where we add the good stuff.

In [325]:
people['KG'] = people['weight'] * KG_TO_LB
people['meters'] = people['height'] * M_TO_IN
people['BMI'] = people['KG'] / people['meters'] ** 2
people['ratio'] = people['meters'] * people['BMI']


In [326]:
# Let's see what we've got

In [327]:
people.shape

(20543, 28)

In [328]:
people.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio'],
      dtype='object')

In [329]:
chadwick.shape

(24258, 8)

In [330]:
# Let's change some of these column names to save ourselves some merging hassle

In [331]:
chadwick.columns

Index(['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref',
       'key_fangraphs', 'mlb_played_first', 'mlb_played_last'],
      dtype='object')

In [332]:
chadwick = chadwick.rename(columns={
    "name_last": "nameLast",
    "name_first": "nameFirst",
    "key_fangraphs": "fangraphsID",
    "key_bbref": "playerID",
    "key_retro": "retroID",
    "key_mlbam": "mlbID"
})

In [333]:
chadwick.columns

Index(['nameLast', 'nameFirst', 'mlbID', 'retroID', 'playerID', 'fangraphsID',
       'mlb_played_first', 'mlb_played_last'],
      dtype='object')

In [334]:
people.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio'],
      dtype='object')

In [335]:
df = people.merge(
    chadwick,
    left_on=["playerID", "retroID", "nameLast", "nameFirst"],
    right_on=["playerID", "retroID", "nameLast", "nameFirst"], how='outer'
)

Merge Chadwick, check.

In [336]:
df.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID',
       'fangraphsID', 'mlb_played_first', 'mlb_played_last'],
      dtype='object')

In [337]:
df.shape

(24587, 32)

prep merge bwar_bat

In [338]:
bwar_bat.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp',
       'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher',
       'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p',
       'runs_replacement', 'runs_above_rep', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def',
       'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG',
       'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent',
       'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off',
       'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_plus', 'TOB_lg', 'TB_lg'],
      dtype='object')

In [339]:
bwar_bat = bwar_bat.rename(columns={
    'player_ID': 'playerID',
    'mlb_ID': 'mlbID',
    'stint_ID': 'stint'
})

In [340]:
bwar_bat.columns

Index(['name_common', 'age', 'mlbID', 'playerID', 'year_ID', 'team_ID',
       'stint', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp',
       'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher',
       'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p',
       'runs_replacement', 'runs_above_rep', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def',
       'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG',
       'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent',
       'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off',
       'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_plus', 'TOB_lg', 'TB_lg'],
      dtype='object')

Merge bwar_bat, check

In [341]:
df = df.merge(
    bwar_bat, left_on=["playerID", "mlbID"], right_on=["playerID", "mlbID"], how="outer"
)

In [342]:
df.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID',
       'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common',
       'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'Inn',
       'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_infield',
       'runs_outfield', 'runs_catcher', 'runs_good_plays', 'runs_defense',
       'runs_position', 'runs_position_p', 'runs_replacement',
       'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off',
       'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def',
       'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG', 'oppRpG',
       'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_

In [343]:
bwar_pitch.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'G', 'GS', 'IPouts', 'IPouts_start',
       'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj',
       'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc',
       'RS_def_total', 'runs_above_avg', 'runs_above_avg_adj',
       'runs_above_rep', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR',
       'salary', 'teamRpG', 'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA',
       'WAA_adj', 'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep',
       'WAR_rep', 'ERA_plus', 'ER_lg'],
      dtype='object')

In [344]:
bwar_pitch = bwar_pitch.rename(columns={
    "stint_ID": "stint",
    "mlb_ID": "mlbID",
    "player_ID": "playerID"
})

In [345]:
bwar_pitch.stint

0        1
1        1
2        1
3        1
4        1
        ..
53650    1
53651    1
53652    1
53653    1
53654    1
Name: stint, Length: 53655, dtype: int64

In [346]:
print(df.columns.tolist())

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher', 'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_p

In [347]:
df = df.merge(
    bwar_pitch,
    left_on=[
        'G',
        'WAA',
        'WAR',
        'WAR_rep',
        'age',
        'lg_ID',
        'mlbID',
        'oppRpG',
        'oppRpG_rep',
        'playerID',
        'pyth_exponent',
        'pyth_exponent_rep',
        'runs_above_avg',
        'runs_above_rep',
        'salary',
        'stint',
        'teamRpG',
        'team_ID',
        'waa_win_perc',
        'waa_win_perc_rep',
        'year_ID'
    ],
    right_on=[
        'G',
        'WAA',
        'WAR',
        'WAR_rep',
        'age',
        'lg_ID',
        'mlbID',
        'oppRpG',
        'oppRpG_rep',
        'playerID',
        'pyth_exponent',
        'pyth_exponent_rep',
        'runs_above_avg',
        'runs_above_rep',
        'salary',
        'stint',
        'teamRpG',
        'team_ID',
        'waa_win_perc',
        'waa_win_perc_rep',
        'year_ID'
    ], how='outer'
)

In [348]:
print(df.columns.tolist())

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common_x', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher', 'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'OPS

In [349]:
fielding_of.columns

Index(['playerID', 'yearID', 'stint', 'Glf', 'Gcf', 'Grf'], dtype='object')

In [350]:
fielding_of = fielding_of.rename(columns={
    "yearID": "year_ID"
})

In [351]:
df = df.merge(
    fielding_of,
    left_on=["playerID", "year_ID", "stint"],
    right_on=["playerID", "year_ID", "stint"], how='left'
)


In [352]:
df.shape

(177727, 104)

In [353]:
fielding.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS',
       'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR'],
      dtype='object')

In [354]:
fielding = fielding.rename(columns={
    "yearID": "year_ID"
})

In [355]:
df = df.merge(
    fielding,
    left_on=["playerID", "year_ID", "G", "stint"],
    right_on=["playerID", "year_ID", "G", "stint"], how='left'
)


In [356]:
df.sample(25) ### works to here

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
36977,goodedw01,1964.0,11.0,16.0,USA,FL,Tampa,NaN,NaN,NaN,NaN,NaN,NaN,Dwight,Gooden,Dwight Eugene,190.0,74.0,R,R,1984-04-07,2000-09-29,goodd001,goodedw01,86.182480,1.8796,24.394286,45.851500,114947.0,1004852.0,1984.0,2000.0,Dwight Gooden,27.0,1992.0,NYM,1.0,NL,77.0,33.0,206.0,-1.28,0.25,0.26,0.0,0.0,0.0,0.0,NaN,0.0,0.16,8.66,0.14,8.200,8.100,8.1,0.2,0.9100,0.89,0.02,0.93,0.02,0.91,0.0200,5166667.0,Y,4.10624,3.86230,0.06979,3.79191,1.807,1.786,0.5276,0.5276,0.5006,0.4918,83.505798,23.601,27.338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140682,garvene01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114640.0,NaN,NaN,NaN,NaN,23.0,1949.0,SLB,1.0,AL,NaN,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.236,21.816,NaN,NaN,2.3001,NaN,NaN,3.99,NaN,NaN,1.8014,NaN,NaN,4.73359,4.19467,NaN,5.17596,1.866,1.923,0.5561,NaN,NaN,0.4572,NaN,NaN,NaN,Ned Garver,32.0,671.0,637.0,34.0,126.0,121.072,0.000,NaN,-17.751,108.0,106.478,147.816,823.0,0.1659,-107.0,22.096,0.203,1.4344,-0.1114,113.975758,112.836,NaN,NaN,NaN,SLA,AL,P,NaN,NaN,15.0,42.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN
17131,chappla01,1890.0,2.0,19.0,USA,IL,McClusky,1918.0,11.0,8.0,USA,CA,San Francisco,Larry,Chappell,LaVerne Ashford,186.0,72.0,L,R,1913-07-18,1917-04-25,chapl101,chappla01,84.368112,1.8288,25.225856,46.133045,112208.0,1002155.0,1913.0,1917.0,Larry Chappell,24.0,1914.0,CHW,1.0,AL,43.0,21.0,NaN,-1.81,0.10,0.00,0.0,NaN,NaN,NaN,NaN,0.0,-0.47,0.00,1.62,-0.600,-2.200,-2.2,-0.5,-0.2700,-0.26,-0.08,-0.09,-0.08,-0.08,0.1800,NaN,N,3.56992,3.67373,0.05821,3.59660,1.758,1.760,0.4874,0.4874,0.4973,0.4907,61.100257,14.126,13.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49791,johnsji04,1983.0,6.0,27.0,USA,NY,Johnson City,NaN,NaN,NaN,NaN,NaN,NaN,Jim,Johnson,James Robert,250.0,78.0,R,R,2006-07-29,2018-09-29,johnj010,johnsji04,113.398000,1.9812,28.890081,57.237028,462382.0,3656.0,2006.0,2018.0,Jim Johnson,34.0,2017.0,ATL,1.0,NL,0.0,56.0,56.7,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000,0.000,0.0,0.0,0.0000,0.00,-0.02,0.00,-0.02,0.00,0.0000,5000000.0,Y,4.66123,4.66123,0.08583,4.66123,1.889,1.889,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31991,fordwh01,1928.0,10.0,21.0,USA,NY,New York,2020.0,10.0,8.0,USA,NY,Lake Success,Whitey,Ford,Edward Charles,178.0,70.0,L,L,1950-07-01,1967-05-21,fordw101,fordwh01,80.739376,1.7780,25.540058,45.410223,114299.0,1004227.0,1950.0,1967.0,Whitey Ford,30.0,1959.0,NYY,1.0,AL,80.0,35.0,204.0,0.03,-0.61,0.34,0.0,0.0,0.0,0.0,NaN,0.0,0.02,9.16,0

In [101]:
teams.sample(10)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1197,1950,NL,SLN,STL,NaN,5,153,76.0,78,75,NaN,NaN,N,N,693,5215,1353,255,50,102,606.0,604.0,23.0,NaN,NaN,NaN,670,598,3.97,57,10,14,4068,1398,119,535,603,130,172,0.978,St. Louis Cardinals,Sportsman's Park IV,1093411.0,104,103,STL,SLN,SLN
1016,1939,AL,NYA,NYY,NaN,1,152,77.0,106,45,NaN,NaN,Y,Y,967,5300,1521,259,55,166,701.0,543.0,72.0,37.0,NaN,NaN,556,496,3.31,87,15,26,4044,1208,85,567,565,126,159,0.978,New York Yankees,Yankee Stadium I,859785.0,99,95,NYY,NYA,NYA
2325,2000,AL,ANA,ANA,W,3,162,81.0,82,80,N,N,N,N,864,5628,1574,309,34,236,608.0,1024.0,93.0,52.0,47.0,43.0,869,805,5.00,5,3,46,4344,1534,228,662,846,134,182,0.978,Anaheim Angels,Edison International Field,2066982.0,102,103,ANA,ANA,ANA
271,1891,AA,LS2,LOU,NaN,8,138,NaN,54,83,NaN,NaN,N,NaN,698,4764,1229,127,68,17,438.0,465.0,227.0,NaN,68.0,NaN,873,567,4.22,126,9,1,3630,1334,32,451,481,454,112,0.922,Louisville Colonels,Eclipse Park I,NaN,97,97,LOU,LS2,LS2
1728,1977,NL,PIT,PIT,E,2,162,81.0,96,66,N,NaN,N,N,734,5662,1550,278,57,133,474.0,878.0,260.0,120.0,34.0,43.0,665,594,3.61,25,15,39,4445,1406,149,485,890,145,137,0.977,Pittsburgh Pirates,Three Rivers Stadium,1237349.0,102,101,PIT,PIT,PIT
2190,1995,AL,DET,DET,E,4,144,72.0,60,84,N,N,N,N,654,4865,1204,228,29,159,551.0,987.0,73.0,36.0,41.0,43.0,844,778,5.49,5,3,38,3825,1509,170,536,729,106,143,0.981,Detroit Tigers,Tiger Stadium,1180979.0,100,101,DET,DET,DET
427,1903,NL,PIT,PIT,NaN,1,141,70.0,91,49,NaN,NaN,Y,N,793,4988,1429,208,110,34,364.0,400.0,172.0,NaN,50.0,NaN,613,405,2.91,117,16,5,3754,1215,9,384,454,295,100,0.951,Pittsburgh Pirates,Exposition Park,326855.0,103,99,PIT,PIT,PIT
2005,1988,AL,KCA,KCR,W,3,161,80.0,84,77,N,NaN,N,N,704,5469,1419,275,40,121,486.0,944.0,137.0,54.0,33.0,51.0,648,580,3.65,29,12,32,4285,1415,102,465,886,124,147,0.980,Kansas City Royals,Royals Stadium,2350181.0,101,101,KCR,KCA,KCA
718,1920,AL,WS1,MIN,NaN,6,153,76.0,68,84,NaN,NaN,N,N,723,5251,1526,233,81,36,433.0,543.0,161.0,114.0,NaN,NaN,802,633,4.17,81,10,10,4101,1521,51,520,418,232,95,0.963,Washington Senators,Griffith Stadium I,359260.0,97,98,WSH,WS1,WS1
468,1906,NL,CIN,CIN,NaN,6,155,78.0,64,87,NaN,NaN,N,N,533,5025,1198,140,71,16,395.0,532.0,170.0,NaN,58.0,NaN,582,409,2.69,126,12,5,4109,1248,14,470,567,262,97,0.959,Cincinnati Reds,Palace of the Fans,330056.0,104,104,CIN,CIN,CIN


Now let's repeat the process with oodles of performance data from df.bwar_bat and df.bwar_pitch.

### Set constants we'll need

### Squish everything into one mondo DF

In [132]:
df_huskiesBatters = df_batters.loc[df_batters.BMI >= 34.55]

In [133]:
df_huskiesBatters.sort_values('BMI').describe()

,birthYear,birthMonth,birthDay,deathYear,deathMonth,deathDay,weight,height,yearID,stint,...,BB,SO,IBB,HBP,SH,SF,GIDP,KG,meters,BMI
count,26.000000,25.000000,25.000000,5.00000,5.000000,5.000000,26.000000,26.000000,26.000000,26.000000,...,26.000000,22.000000,0.0,24.000000,17.000000,0.0,1.0,26.000000,26.000000,26.000000
mean,1969.153846,5.800000,17.520000,1952.40000,7.600000,8.600000,269.961538,72.115385,1901.730769,1.115385,...,9.884615,14.909091,NaN,0.875000,6.000000,NaN,0.0,122.452394,1.831731,36.408324
std,40.878055,3.316625,8.529947,49.45503,3.646917,7.602631,30.180100,4.348121,14.177610,0.325813,...,16.310308,13.606403,NaN,1.650099,8.951257,NaN,NaN,13.689452,0.110442,1.728873
min,1853.000000,1.000000,1.000000,1891.00000,2.000000,2.000000,155.000000,55.000000,1872.000000,1.000000,...,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,70.306760,1.397000,34.622243
25%,1977.000000,4.000000,11.000000,1915.00000,6.000000,2.000000,261.250000,71.000000,1890.250000,1.000000,...,1.000000,3.250000,NaN,0.000000,0.000000,NaN,0.0,118.500910,1.803400,34.891189
50%,1983.500000,6.000000,18.000000,1966.00000,9.000000,6.000000,270.000000,72.000000,1903.500000,1.000000,...,3.500000,11.000000,NaN,0.000000,2.000000,NaN,0.0,122.469840,1.828800,35.875979
75%,1989.000000,8.000000,24.000000,1975.00000,10.000000,14.000000,283.750000,75.000000,1912.000000,1.000000,...,9.250000,25.250000,NaN,1.000000,7.000000,NaN,0.0,128.706730,1.905000,37.928366
max,1998.000000,12.000000,30.000000,2015.00000,11.000000,19.000000,320.000000,78.000000,1924.000000,2.000000,...,68.000000,43.000000,NaN,6.000000,33.000000,NaN,0.0,145.149440,1.981200,40.292666


In [134]:
df_huskiesPitchers = df[df_pitchers.BMI > 34.55]

NameError: name 'df' is not defined

In [ ]:
df_huskiesPitchers.head()

In [ ]:
df_huskies = pd.merge(df_huskiesBatters, df_huskiesPitchers, how='right', on='playerID')

In [ ]:
df_huskies.columns

In [ ]:
sns.jointplot(data=df_simple, x="height", y="weight", kind = "reg", truncate = False)

In [ ]:
sns.choose_diverging_palette()

In [ ]:

# Compute the correlation matrix
corr = df_huskiesBatters.corr(method="spearman")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(290, 10, n=40, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    center=0,
    square=True,
    linewidths=0.25,
    cbar_kws={"shrink": .5},
)



In [ ]:
corr_mat = df.corr().stack().reset_index(name="correlation")